# CHAPTER 8. 비동기 I/O

## 목차

* [8.1 비동기 프로그래밍 소개](#8.1-비동기-프로그래밍-소개)
* [8.2 async/await의 동작 방식](#8.2-async/await의-동작-방식)
* [8.3 CPU 공유 I/O 부하](#8.3-CPU-공유-I/O-부하)
* [8.4 마치며](#8.4-마치며)

## 레퍼런스
* [Context Switching](https://philip1994.tistory.com/45)
* [Concurrency](https://m.blog.naver.com/hse05105/221692340386)
* [Concurrency - Lock](https://datacadamia.com/data/concurrency/lock)
* [Mutual Exclusion in Distributed System ](https://www.geeksforgeeks.org/mutual-exclusion-in-distributed-system/)

## 8.1 비동기 프로그래밍 소개

> 크게 CPU 통한 연산과 커널을 통한 I/O 를 통해 대부분의 애플리케이션이 동작하게 되는데, 두 가지 동작이 순차적으로 혹은 번갈아가면서 수행되는데, 예를 들어 네트워크 소켓에 데이터를 쓰는 연산은 보통 1ms 가 걸리는데, 2.4GHz CPU는 같은 시간에 명령어 240만 개를 처리할 수 있습니다. 

> 이렇듯 I/O 연산이 CPU 처리보다 상당히 느리기 때문에 이러한 `I/O Wait` 상태가 자주 발생하지 않는 것이 애플리케이션 성능을 높이는 데에 큰 도움을 줄 수 있습니다.


### `Context Switching` 이란?

> **쓰레드는 O/S에서의 기본적인 스케줄링 단위**이다. 그리고 이러한 쓰레드를 O/S는 여러 개를 관리하면서 우리에게 멀티 태스킹을 가능하게 해준다. 하지만 앞서 ***일반적인 CPU는 한번에 한가지 명령만을 계산할 수 있다***고 하였다. 단지 여러 쓰레드의 순차적인 관리를 통해 거의 동시에 진행하게끔 한다고 하였는데, 이러한 관리를 위해서 *쓰레드를 O/S에서 관리할 수 있는 기본적인 정보*가 있어야 한다.

> **쓰레드는 레지스터(Register Set), 커널 스택(Kernel Stack), 사용자 스택 등의 여러 정보를 갖고 있는데, 이러한 정보들을 Context**라고 한다. Context들은 쓰레드가 작업을 진행하는 동안 작업에 대한 정보를 보관하고 있으며, *다른 쓰레드로 작업 순서가 넘어갈 때 저장*된다. O/S는 쓰레드 하나의 작업을 진행하기 위해 그 쓰레드의 Context를 읽어오며, 다시 다른 쓰레드로 작업을 변경할 때 이전 쓰레드의 Context를 저장하고 다시 진행할 쓰레드의 Context를 읽어오는 작업을 반복하게된다. 이처럼 **한 쓰레드에서 다른 쓰레드로 작업을 넘기는 과정을 Context Switching**이라고 한다.

* CPU 의 스케줄링 단위인 *Thread 간의 처리 전환* 혹은 *I/O 와의 처리 전환*을 위한 **Register, Stack 등의 정보를 저장해 두었다가 재개 시에 다시 CPU 수준에서 읽어오는 과정**을 `Context Switching`이라고 합니다



### Partial 함수와 Queue 클래스를 통한 Eventloop 구현을 통해 Nonblocking 함수 호출 실습

* 일반적으로 동시성 프로그램은 **이벤트 루프**를 사용하는데, 실행할 함수의 목록을 관리하는 것이라고 말할 수 있습니다.
* [functools.partial](https://docs.python.org/3/library/functools.html#functools.partial) 매개변수를 포함한 함수를 만들 수 있는 함수
* 아래의 예제에서는 Queue 상속을 통해 eventloop 를 구현하고, functools 의 함수 전달을 통해 nonblocking 호출을 합니다

> 함수의 호출 또한 Context Switching 비용이 들지만, I/O 대기 시간을 활용해서 얻는 이익이 훨씬 크기 떄문에 **I/O 대기가 많은 상황에서 동시성의 이익이 극대화** 된다는 점을 기억해 두어야 합니다.

In [10]:
import sys
from queue import Queue
from functools import partial

eventloop = None
iteration = 11

class EventLoop(Queue):
    def start(self):
        global iteration
        while True:
            function = self.get()
            iteration -= 1
            if (iteration == 0):
                break
            function()

def do_hello():
    global eventloop
    sys.stdout.write("Hello ")
    eventloop.put(do_world) # do_world 함수를 넣기만 하고, 바로 다음 작업을 진행

def do_world():
    global eventloop
    print("World")
    eventloop.put(do_hello) # do_hello 함수를 넣기만 하고, 바로 다음 작업을 진행
    
if __name__ == "__main__":
    eventloop = EventLoop()
    eventloop.put(do_hello) # do_hello 함수를 호출을 시키고, 바로 다음 명령 수행 (async)
    eventloop.start()

Hello World
Hello World
Hello World
Hello World
Hello World


### Partial 함수와 Callback 함수의 Eventloop 구현을 통해 Nonblocking 함수 호출 실습

In [16]:
from functools import partial
import time

eventloop = None

class EventLoop(Queue):
    def start(self):
        global iteration
        while True:
            if self.qsize() == 0: break
            function = self.get()
            function()

# async 함수 구현을 위해 eventloop 를 활용합니다
def save_result_to_db(value, callback):
    global eventloop
    result = f"Return {value} + modified value"
    print("running database processes")
    eventloop.put(partial(time.sleep, 5))
    eventloop.put(partial(callback, result))

def save_value(value, callback):
    print(f"Saving {value} to database")
    save_result_to_db(value, callback)
    
def print_response(db_response):
    print(f"Response from database: {db_response}")
    
if __name__ == "__main__":
    eventloop = EventLoop()
    eventloop.put(partial(save_value, "partial + callback style", print_response))
    eventloop.start()

Saving Hello World to database
running database processes
Response from database: Return Hello World + modified value


### async 및 await 키워드를 통해 callback 방식을 future 방식으로 변경

> 파이썬 3.4 이전에는 콜백 패러다임이 많이 활용되었으나, asyncio 표준 라이브러리 모듀과 PEP492 가 퓨처 메커니즘을 파이썬 네이티브로 만들면서 `await` 와 `async` 키워드를 통해 **비동기 함수를 정의**할 수 있게 되었습니다 

#### [asyncio.createTask](https://docs.python.org/3/library/asyncio-task.html#asyncio.create_task)

> 기존의 callback 방식과의 가장 큰 차이점은 callback 방식은 제어권이 호출하는 함수에 있지않고, 해당 callback 함수에서만 처리할 수 밖에 없어 불편함이 있으며 계속 코드를 따라다니면서 이해해야 하는 불편함이 있다

> 반면에 future 방식은 코드의 제어권이 호출하는 함수에 있어 반환값을 그대로 사용할 수 있어 코드가 직관적이다

In [62]:
from functools import partial
import time
import asyncio

async def save_result_to_db(value):
    global eventloop
    print("running database processes")
    await asyncio.sleep(5)
    result = f"Return {value} + modified value"
    print(result + " is returned.")
    return result

async def save_value(value):
    print(f"Saving {value} to database")
    futures, _= await asyncio.wait([
        save_result_to_db(value)
    ])
    for future in futures:
        if (future.get_name() == "foo"):
            print_response(future.result())
    
def print_response(db_response):
    print(f"Response from database: {db_response}")
    
if __name__ == "__main__":
    await asyncio.create_task(save_value("async + await + future style"), name="foo")

Saving async + await + future style to database
running database processes
Return async + await + future style + modified value is returned.


* [맨 위로](#CHAPTER-8.-비동기-I/O)

## 8.2 async/await의 동작 방식

### 8.2.1 순차적 크롤러


### 8.2.2 gevent

### 8.2.3 tornado


### 8.2.4 aiohttp

* [맨 위로](#CHAPTER-8.-비동기-I/O)

## 8.3 CPU 공유 I/O 부하

### 8.3.1 순차처리

### 8.3.2 일괄처리

* [맨 위로](#CHAPTER-8.-비동기-I/O)

## 8.4 마치며

* [맨 위로](#CHAPTER-8.-비동기-I/O)
